<a href="https://colab.research.google.com/github/xpandalord/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [0]:
### YOUR CODE STARTS HERE
# imports
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import pandas as pd
import numpy as np

In [2]:
# get the data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-05-21 00:39:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.3’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.07s   

2020-05-21 00:39:07 (270 KB/s) - ‘house-votes-84.data.3’ saved [18171/18171]



In [53]:
# make into a dataframe
column_headers = ['Class Name', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools',
                  'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending', 'superfund-right-to-sue', 'crime',
                  'duty-free-exports', 'export-administration-act-south-africa']
df = pd.read_csv('house-votes-84.data', header = None, names = column_headers, na_values = '?')
df.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [24]:
# recode votes as numeric
df = df.replace({'y': 1, 'n': 0})
df.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [5]:
# how many from each party?
df['Class Name'].value_counts()

democrat      267
republican    168
Name: Class Name, dtype: int64

In [6]:
# how did Republicans vote?
rep = df[df['Class Name']=='republican']
rep.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [7]:
# how did Democrats vote?
dem = df[df['Class Name']=='democrat']
dem.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [8]:
# the percentage of republicans who voted "yes" (1) 
# on the handicapped-infants bill

rep['handicapped-infants'].sum()/len(rep)

# len() is counting NaN values too!

0.18452380952380953

In [9]:
# Remove NaN values from this column

col = rep['handicapped-infants']

np.isnan(col)

handicapped_infants_no_nans = col[~np.isnan(col)]

# The same column as before, but I've dropped he NaN values
handicapped_infants_no_nans

handicapped_infants_no_nans.sum()/len(handicapped_infants_no_nans)

0.18787878787878787

In [10]:
# Average rate of voting 'yes' on the handicapped-infants
rep['handicapped-infants'].mean()

0.18787878787878787

1. Null Hypothesis: There is no difference between average voting rates (levels of support) for the adoption of the budget resolution between democrats and republicans in the house of representatives. (support is equal)
> x_1 = x_2

 Where x_1 is the mean of the democrat votes and x_2 is the mean of the republican votes.

2. Alternative Hypothesis:
>x_1 != x_2

 Levels of support between the two parties will differ.

3. 99% Confidence Level

In [0]:
from scipy.stats import ttest_ind

In [13]:
# What is the mean support of Democrats?
dem['adoption-of-the-budget-resolution'].mean()

0.8884615384615384

In [12]:
# What is the mean support of Republicans?
rep['adoption-of-the-budget-resolution'].mean()

0.13414634146341464

In [14]:
# compare with a t-test:
ttest_ind(dem['adoption-of-the-budget-resolution'], rep['adoption-of-the-budget-resolution'])

Ttest_indResult(statistic=nan, pvalue=nan)

In [18]:
# account for NaN's
ttest_ind(dem['adoption-of-the-budget-resolution'], rep['adoption-of-the-budget-resolution'], nan_policy = 'omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

In [16]:
# You could also remove NaN values from this column

col = dem['adoption-of-the-budget-resolution']
dem_adoption_of_the_budget_resolution_no_nans = col[~np.isnan(col)]

col = rep['adoption-of-the-budget-resolution']
rep_adoption_of_the_budget_resolution_no_nans = col[~np.isnan(col)]

# My sample sizes for the two samepls:
print(len(dem_adoption_of_the_budget_resolution_no_nans))
print(len(rep_adoption_of_the_budget_resolution_no_nans))

260
164


When I have multiple samples (using a 2-sample t-test) I will use the smaller of the two samples to determine my degrees of freedom.

So in this case, df = 164 - 1 = 163

4. T-statistic: 23.213
5. P-value: 2.070e^-77

I want to reject the null hypothesis if my p-value is < 0.01 or if my p-value is less than (1 - confidence_level)

Conclusion: Due to a p-value of 2.070e^-77, I reject the null hypothesis that the democrat and republican support for the adoption of the budget resolution is the same. Since the democrats have a greater mean than the republicans, then the democrats support the adoption of the budget resolution more than the republicans with p < 0.01.

1. Null Hypothesis: There is no difference between average voting rates (levels of support) for the physician fee freeze between republicans and democrats in the house of representatives. (support is equal)
> x_1 = x_2

 Where x_1 is the mean of the republican votes and x_2 is the mean of the democrat votes.

2. Alternative Hypothesis:
>x_1 != x_2

 Levels of support between the two parties will differ.

3. 99% Confidence Level

In [19]:
# What is the mean support of Republicans?
rep['physician-fee-freeze'].mean()

0.9878787878787879

In [20]:
# What is the mean support of Democrats?
dem['physician-fee-freeze'].mean()

0.05405405405405406

In [22]:
# compare with a t-test:
ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy = 'omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

4. T-statistic: 49.367
5. P-value: 1.994e^-177

Conclusion: Due to a p-value of 1.994e^-177, I reject the null hypothesis that the republican and democrat support for the physician fee freeze is the same. Since the republicans have a greater mean than the democrats, then the republicans support the physician fee freeze more than the democrats with p < 0.01.

1. Null Hypothesis: There is no difference between average voting rates (levels of support) for the water project cost sharing between republicans and democrats in the house of representatives. (support is equal)
> x_1 = x_2

 Where x_1 is the mean of the republican votes and x_2 is the mean of the democrat votes.

2. Alternative Hypothesis:
>x_1 != x_2

 Levels of support between the two parties will differ.

3. 90% Confidence Level

In [45]:
# What is the mean support of Republicans?
rep['water-project-cost-sharing'].mean()

0.5067567567567568

In [46]:
# What is the mean support of Democrats?
dem['water-project-cost-sharing'].mean()

0.502092050209205

In [48]:
# compare with a t-test:
ttest_ind(rep['water-project-cost-sharing'], dem['water-project-cost-sharing'], nan_policy = 'omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

4. T-statistic: 0.0890
5. P-value: 0.929

Conclusion: Due to a p-value of 0.929, I fail to reject the null hypothesis that the republican and democrat support for the water project cost sharing is the same.

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!

In [0]:
# ttest function
def ttest(column):
  return ttest_ind(rep[column], dem[column], nan_policy = 'omit')

In [81]:
# a test checking if the function works
ttest('handicapped-infants')

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)